In [7]:
# HIDDEN

# from IPython.display import HTML 
# HTML("""<script>code_show=true; function code_toggle() { if (code_show){ $('div.input').hide(); } else { $('div.input').show(); } code_show = !code_show} $( document ).ready(code_toggle);</script><form action="javascript:code_toggle()"><input type="submit" value=""></form>""" )

In [2]:
# HIDDEN
# !pip install vpython
from vpython import *
import ipywidgets as wd
import numpy as np

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
# HIDDEN
def arrow_head(width=1, height=2, inset=0, rot=0):
    top_point = (0,height,0)
    bottom_point = (0,inset,0)
    left_point = (-width/2, 0, 0)
    right_point = (width/2, 0, 0)
    arrow_shape = shapes.points(pos=[bottom_point,left_point,top_point, right_point], 
                         rotate=rot*pi/2)
    return arrow_shape

In [4]:
scene = canvas()

<IPython.core.display.Javascript object>

In [5]:
# HIDDEN
shaft_diam = wd.FloatSlider(value=2.375, min=0.5, max=4, step=0.125, continuous_update=False, description='diam (in)', readout_foramat='%1.3f')
shaft_thickness = wd.FloatSlider(value=3, min=1, max=8, step=1, continuous_update=False, description='depth (1/8in)')
sheet_thickness = wd.FloatSlider(value=2, min=1, max=8, step=1, continuous_update=False, description='thick (1/8in)')

shaft_length = wd.FloatSlider(value=10, min=13, max=16, step=0.25, continuous_update=False, description='length (ft)')
head_position = wd.FloatSlider(value=0, min=0, max=3, step=0.25, continuous_update=False, description='head (ft)')
tail_position = wd.FloatSlider(value=0, min=0, max=5, step=0.25, continuous_update=False, description='tail (ft)')

arrow_width = wd.FloatSlider(value=1.5, min=0.5, max=3, step=0.25, continuous_update=False, description='head width')
arrow_height = wd.FloatSlider(value=3, min=0.5, max=5, step=0.25, continuous_update=False, description='head height')
arrow_inset = wd.FloatSlider(value=0.5, min=0, max=3, step=0.25, continuous_update=False, description='head inset')

tail_width = wd.FloatSlider(value=1, min=0.5, max=3, step=0.25, continuous_update=False, description='tail width')
tail_height = wd.FloatSlider(value=2, min=0.5, max=5, step=0.25, continuous_update=False, description='tail height')

run_button = wd.Button(description="Run!")

scene.background = color.gray(0.8)
scene.forward = vec(0,-0.2,-1)
scene.fov = 0.2
scene.range = 3.8
scene.caption = """Right button drag or Ctrl-drag to rotate "camera" to view scene.
To zoom, drag with middle button or Alt/Option depressed, or use scroll wheel.
     On a two-button mouse, middle is left + right.
Shift-drag to pan left/right and up/down.
Touch screen: pinch/extend to zoom, swipe or two-finger rotate.\n"""


def update(*args):
    for obj in scene.objects:
        obj.visible = False
        del obj
    tube = extrusion(path=[vec(0,0,0), vec(shaft_length.value,0,0)], 
                 shape=shapes.circle(radius=0.5*shaft_diam.value/12, 
                                     thickness=(shaft_thickness.value/8)/12),
                 pos=vec(0,0,0), axis=vec(1,0,0), 
                 color=color.yellow, 
                 end_face_color=color.blue)

    
    metal_width = sheet_thickness.value/12/8 # convert to feet 
    arrow_wd, arrow_ht = arrow_width.value, arrow_height.value
    ns = arrow_inset.value
    arrow_depth, tail_depth = head_position.value, tail_position.value
    arrow_shape = arrow_head(width=arrow_wd, height=arrow_ht, inset=ns)
    head = extrusion(path=[vec(0,0,0), vec(metal_width,0,0)],
                    shape=arrow_shape,
                    axis=vec(0,1,0), 
                    pos=vec(shaft_length.value/2+arrow_ht/2 - arrow_depth,0,0),
                    texture=textures.metal
                   )
    
    tail_wd, tail_ht = tail_width.value, tail_height.value
    n=100
    sinpath=[vec(np.sin((i/n)*2*np.pi),0,0) for i in range(n)]
    #     tail = extrusion(path=[vec(0,0,0), vec(metal_width,0,0)],
    tail = extrusion(path=sinpath,
                    shape=shapes.rectangle(height=tail_ht, width=tail_wd),
                    axis=vec(0,1,0), 
                    pos=vec(-shaft_length.value/2-tail_ht/2 + tail_depth,0,0),
                   )


scene.waitfor('textures')

run_button.on_click(update)

In [6]:
wd.HBox([
    wd.VBox([shaft_length, shaft_diam, shaft_thickness, sheet_thickness]),
    wd.VBox([head_position, arrow_width, arrow_height, arrow_inset]),
    wd.VBox([tail_position, tail_width, tail_height, run_button])
    ])